In [ ]:
# Libreria di sistema
import kagglehub
from pathlib import Path
import os.path

# Installa dipendenze base
!pip install torch torchvision
!pip install pandas numpy matplotlib seaborn
!pip install scikit-learn pillow tqdm
!pip install jupyter notebook
!pip install tensorboard

In [ ]:
# Download dataset
path = kagglehub.dataset_download("rtlmhjbn/ip02-dataset")

print("Path to dataset files:", path)

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
from collections import Counter
import json

# Configurazione visualizzazioni
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Analisi Distribuzioni Classi

In [ ]:
DATA_DIR = Path('/kaggle/input/ip02-dataset/classification')
TRAIN_DIR = DATA_DIR / 'train'
VAL_DIR = DATA_DIR / 'val'
TEST_DIR = DATA_DIR / 'test'

print(f"Train directory exists: {TRAIN_DIR.exists()}")
print(f"Val directory exists: {VAL_DIR.exists()}")
print(f"Test directory exists: {TEST_DIR.exists()}")

def count_images_per_class(data_dir):
    class_counts = {}
    
    for class_folder in sorted(data_dir.iterdir()):
        if class_folder.is_dir():
            class_name = class_folder.name
            num_images = len(list(class_folder.glob('*.jpg'))) + \
                        len(list(class_folder.glob('*.png'))) + \
                        len(list(class_folder.glob('*.jpeg')))
            class_counts[class_name] = num_images
    
    return class_counts

# Conta immagini per split
train_counts = count_images_per_class(TRAIN_DIR)
val_counts = count_images_per_class(VAL_DIR) if VAL_DIR.exists() else {}
test_counts = count_images_per_class(TEST_DIR) if TEST_DIR.exists() else {}

# Crea DataFrame
df_train = pd.DataFrame(list(train_counts.items()), columns=['Class', 'Train_Count'])
if val_counts:
    df_val = pd.DataFrame(list(val_counts.items()), columns=['Class', 'Val_Count'])
    df_train = df_train.merge(df_val, on='Class', how='outer')
if test_counts:
    df_test = pd.DataFrame(list(test_counts.items()), columns=['Class', 'Test_Count'])
    df_train = df_train.merge(df_test, on='Class', how='outer')

df_train = df_train.fillna(0).astype({'Train_Count': int})

print("STATISTICHE DATASET")
print(f"Numero totale di classi: {len(train_counts)}")
print(f"Totale immagini train: {sum(train_counts.values())}")
if val_counts:
    print(f"Totale immagini val: {sum(val_counts.values())}")
if test_counts:
    print(f"Totale immagini test: {sum(test_counts.values())}")


print("STATISTICHE TRAIN SET")
print(f"Media immagini per classe: {np.mean(list(train_counts.values())):.1f}")
print(f"Mediana: {np.median(list(train_counts.values())):.1f}")
print(f"Min immagini: {min(train_counts.values())} (classe: {min(train_counts, key=train_counts.get)})")
print(f"Max immagini: {max(train_counts.values())} (classe: {max(train_counts, key=train_counts.get)})")
print(f"Std deviation: {np.std(list(train_counts.values())):.1f}")

# Calcola imbalance ratio
imbalance_ratio = max(train_counts.values()) / min(train_counts.values())
print(f"\n IMBALANCE RATIO: {imbalance_ratio:.2f}x")
if imbalance_ratio > 10:
    print("SEVERAMENTE SBILANCIATO - Necessarie tecniche di bilanciamento!")
elif imbalance_ratio > 5:
    print("MODERATAMENTE SBILANCIATO - Consigliato class weighting")
else:
    print("RELATIVAMENTE BILANCIATO")

# Visualizzazione Distribuzione

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(16, 12))

# Plot 1: Distribuzione ordinata
sorted_counts = dict(sorted(train_counts.items(), key=lambda x: x[1], reverse=True))
axes[0].bar(range(len(sorted_counts)), list(sorted_counts.values()), color='steelblue', alpha=0.7)
axes[0].axhline(np.mean(list(train_counts.values())), color='red', linestyle='--', 
                label=f'Media: {np.mean(list(train_counts.values())):.0f}', linewidth=2)
axes[0].set_xlabel('Classe (ordinata per frequenza)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Numero di Immagini', fontsize=12, fontweight='bold')
axes[0].set_title('Distribuzione Classi - Train Set (Ordinato)', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(axis='y', alpha=0.3)

# Plot 2: Istogramma distribuzione
axes[1].hist(list(train_counts.values()), bins=30, color='coral', alpha=0.7, edgecolor='black')
axes[1].axvline(np.mean(list(train_counts.values())), color='red', linestyle='--', 
                label=f'Media: {np.mean(list(train_counts.values())):.0f}', linewidth=2)
axes[1].axvline(np.median(list(train_counts.values())), color='green', linestyle='--', 
                label=f'Mediana: {np.median(list(train_counts.values())):.0f}', linewidth=2)
axes[1].set_xlabel('Numero di Immagini per Classe', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Frequenza (numero di classi)', fontsize=12, fontweight='bold')
axes[1].set_title('Distribuzione della Frequenza delle Classi', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Identifica classi problematiche
threshold_low = np.percentile(list(train_counts.values()), 25)
threshold_high = np.percentile(list(train_counts.values()), 75)

underrepresented = {k: v for k, v in train_counts.items() if v < threshold_low}
overrepresented = {k: v for k, v in train_counts.items() if v > threshold_high}

print(f"\n📊 Classi sotto-rappresentate (<25° percentile = {threshold_low:.0f} img): {len(underrepresented)}")
print(f"📊 Classi sovra-rappresentate (>75° percentile = {threshold_high:.0f} img): {len(overrepresented)}")


# Visualizzazione Dataset

In [ ]:
def show_sample_images(data_dir, n_classes=6, n_images_per_class=5):
    # Mostra immagini di esempio da diverse classi

    classes = sorted([d.name for d in data_dir.iterdir() if d.is_dir()])
    selected_classes = np.random.choice(classes, min(n_classes, len(classes)), replace=False)
    
    fig, axes = plt.subplots(n_classes, n_images_per_class, 
                             figsize=(3*n_images_per_class, 3*n_classes))
    fig.suptitle('Esempi di Immagini dal Dataset', fontsize=16, fontweight='bold', y=0.995)
    
    for i, class_name in enumerate(selected_classes):
        class_dir = data_dir / class_name
        images = list(class_dir.glob('*.jpg')) + list(class_dir.glob('*.png'))
        selected_images = np.random.choice(images, min(n_images_per_class, len(images)), replace=False)
        
        for j, img_path in enumerate(selected_images):
            img = Image.open(img_path)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_ylabel(f'{class_name}\n({train_counts[class_name]} img)', 
                                     fontsize=10, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

show_sample_images(TRAIN_DIR, n_classes=6, n_images_per_class=5)


In [ ]:
def calculate_class_weights(class_counts):
    # Calcola pesi per bilanciare le classi
    total = sum(class_counts.values())
    n_classes = len(class_counts)
    
    # Formula: weight = total / (n_classes * class_count)
    weights = {cls: total / (n_classes * count) for cls, count in class_counts.items()}
    
    return weights

class_weights = calculate_class_weights(train_counts)

# Visualizza top-10 pesi
sorted_weights = dict(sorted(class_weights.items(), key=lambda x: x[1], reverse=True)[:50])
print("\n" + "="*80)
print("TOP 10 CLASS WEIGHTS (per loss function)")
print("="*80)
for cls, weight in sorted_weights.items():
    print(f"{cls:30s}: {weight:.4f} (n_images={train_counts[cls]})")

# Salva per uso futuro
weights_dict = {
    'class_weights': class_weights,
    'class_counts': train_counts
}

print(" EDA COMPLETATA - Prossimo step: implementare data loader e baseline model")

In [ ]:
pip install albumentations

# Preparazione Dataset e Bilanciamento

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from PIL import Image
import os
import cv2
from pathlib import Path
import json
import numpy as np
from typing import Optional, Dict, Tuple
import albumentations as A
from albumentations.pytorch import ToTensorV2


class IP102Dataset(Dataset):
    
    def __init__(
        self,
        root_dir: str,
        transform=None,
        resize_limit=1024,
        augment: bool = True,
        cache_images: bool = False
    ):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.augment = augment
        self.cache_images = cache_images
        self.resize_limit = resize_limit
        
        # Carica immagini e labels
        self.samples = []
        self.class_to_idx = {}
        self.idx_to_class = {}
        self._load_dataset()
        
        self.image_cache = {} if cache_images else None
        
        print(f"Dataset caricato: {len(self.samples)} immagini, {len(self.class_to_idx)} classi")
    
    def _load_dataset(self):
        
        classes = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(classes)}
        self.idx_to_class = {idx: cls_name for cls_name, idx in self.class_to_idx.items()}
        
        for class_name in classes:
            class_dir = self.root_dir / class_name
            class_idx = self.class_to_idx[class_name]
            
            # Trova tutte le immagini
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']:
                for img_path in class_dir.glob(ext):
                    self.samples.append((str(img_path), class_idx))
    
    def get_class_distribution(self) -> Dict[int, int]:
        # Restituisce distribuzione delle classi
        class_counts = {}
        for _, label in self.samples:
            class_counts[label] = class_counts.get(label, 0) + 1
        return class_counts
    
    def __len__(self):
        return len(self.samples)

    def _fast_load(self, path):
        img = cv2.imread(path, cv2.IMREAD_COLOR)

        # in caso di errore di lettura
        if img is None:
            return np.zeros((224, 224, 3), dtype=np.uint8)

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # immagini IP102 molto grandi → riduzione preventiva
        h, w = img.shape[:2]
        max_dim = max(h, w)
        if max_dim > self.resize_limit:
            scale = self.resize_limit / max_dim
            img = cv2.resize(img, (int(w * scale), int(h * scale)))

        return img
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = self._fast_load(img_path)
        
        # Applica trasformazioni
        if self.transform:
            img = self.transform(image=img)["image"]
        
        return img, label

def get_train_transforms(img_size: int = 224):

    return A.Compose([

        A.RandomResizedCrop(size=(img_size, img_size),scale=(0.7, 1.0),ratio=(0.75, 1.33),p=1.0),
        
        # Augmentazioni leggere e realistiche
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.25),

        A.Rotate(limit=25, p=0.5),
        
        A.ColorJitter(
            brightness=0.2, contrast=0.2, saturation=0.2, hue=0.05,
            p=0.5
        ),

        # leggerissimo rumore
        A.GaussNoise(var_limit=(5.0, 15.0), p=0.15),

        # piccolo CutOut
        A.CoarseDropout(
            max_holes=6,
            max_height=20,
            max_width=20,
            min_holes=1,
            fill_value=0,
            p=0.2
        ),

        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
        ToTensorV2()
    ])

def get_val_transforms(img_size=224):
    
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])


def create_weighted_sampler(dataset: IP102Dataset) -> WeightedRandomSampler:

    class_counts = dataset.get_class_distribution()
    
    # Calcola pesi: inversamente proporzionale alla frequenza
    total_samples = len(dataset)
    num_classes = len(class_counts)
    class_weights = {
        cls: total_samples / (num_classes * count) 
        for cls, count in class_counts.items()
    }
    
    # Assegna peso a ogni sample
    sample_weights = [class_weights[label] for _, label in dataset.samples]
    
    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    return sampler


def get_dataloaders(
    train_dir: str,
    val_dir: str,
    batch_size: int = 32,
    img_size: int = 224,
    num_workers: int = 4,
):
    
    # Crea transforms
    train_transform = get_train_transforms(img_size)
    val_transform = get_val_transforms(img_size)
    
    # Crea datasets
    train_dataset = IP102Dataset(train_dir, transform=train_transform)
    val_dataset = IP102Dataset(val_dir, transform=val_transform)
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )
    
    print(f" Train samples: {len(train_dataset)}")
    print(f" Val samples:   {len(val_dataset)}")
    
    return train_loader, val_loader

# Visualizzazione immagini dal nuovo DataLoader IP102

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def denormalize(img_tensor):
    img = img_tensor.cpu().numpy().transpose(1, 2, 0)

    mean = np.array([0.485, 0.456, 0.406])
    std  = np.array([0.229, 0.224, 0.225])

    img = img * std + mean
    img = np.clip(img, 0, 1)
    return img


TRAIN_DIR = "/kaggle/input/ip02-dataset/classification/train"
VAL_DIR   = "/kaggle/input/ip02-dataset/classification/val"

train_loader, val_loader = get_dataloaders(
    train_dir=TRAIN_DIR,
    val_dir=VAL_DIR,
    batch_size=16,
    img_size=224,
    num_workers=4
)


print("\n Carico un batch dal DataLoader…\n")
images, labels = next(iter(train_loader))

print(" Batch images:", images.shape)
print(" Batch labels:", labels.shape)
print(" Example labels:", labels[:10].tolist())

plt.figure(figsize=(14, 10))
plt.suptitle("Esempi di immagini augmentate (train_loader)", fontsize=16, y=1.02)

for i in range(12):  # mostra 12 immagini
    img = denormalize(images[i])

    plt.subplot(3, 4, i+1)
    plt.imshow(img)
    plt.title(f"Class {labels[i].item()}")
    plt.axis("off")

plt.tight_layout()
plt.show()

In [ ]:
pip install timm

In [ ]:
from pathlib import Path

path_checkpoints = Path("/kaggle/working/checkpoints/Efm")
path_logs = Path("/kaggle/working/logs/Efm")

path_checkpoints.mkdir(parents=True, exist_ok=True)
path_logs.mkdir(parents=True, exist_ok=True)

if path_checkpoints.exists() and path_checkpoints.is_dir() and path_logs.exists() and path_logs.is_dir():
    print("Cartelle trovate")
else:
    print("Cartelle non trovate")

In [ ]:
from pathlib import Path

path_checkpoints_baseline = Path("/kaggle/working/checkpoints/baseline")
path_logs_baseline = Path("/kaggle/working/logs/baseline")

path_checkpoints_baseline.mkdir(parents=True, exist_ok=True)
path_logs_baseline.mkdir(parents=True, exist_ok=True)

if path_checkpoints_baseline.exists() and path_checkpoints_baseline.is_dir() and path_logs_baseline.exists() and path_logs_baseline.is_dir():
    print("Cartelle trovate")
else:
    print("Cartelle non trovate")

# Esplorazione Modelli EfficientNet (timm)

In [ ]:
import timm
# Lista tutti i modelli EfficientNet disponibili
efficientnet_models = timm.list_models('*efficientnet*', pretrained=True)
print("EfficientNet models con pretrained weights:")
for model in efficientnet_models[:50]:  # Mostra primi 50
    print(f"  - {model}")

In [ ]:
import timm
m = timm.create_model('efficientnetv2_rw_m.agc_in1k', pretrained=True)
print(m.default_cfg['input_size'])

# Training Pipeline e Ottimizzazione

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR
import timm
from tqdm import tqdm
import numpy as np
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, 
    f1_score, confusion_matrix
)
import matplotlib.pyplot as plt
import torch.nn.functional as F
import seaborn as sns
import json
from copy import deepcopy
from pathlib import Path
import time
import random
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

import torchvision.transforms as T
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, WeightedRandomSampler

def mixup_data(x: torch.Tensor, y: torch.LongTensor, alpha: float = 0.3):

    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size, device=x.device)

    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def cutmix_data(x: torch.Tensor, y: torch.LongTensor, alpha: float = 0.5):

    
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0

    batch_size, _, W, H = x.size()
    index = torch.randperm(batch_size, device=x.device)

    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    lam = 1.0 - ((bbx2 - bbx1) * (bby2 - bby1) / (W * H))
    
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


class EMA:
    def __init__(self, model: nn.Module, decay: float = 0.9999):
        self.decay = decay
        self.model = model
        self.shadow = {}
        self.backup = {}
        
        for name, param in model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    @torch.no_grad()
    def update(self):
        for name, param in self.model.named_parameters():
            if not param.requires_grad:
                continue
            new_avg = self.decay * self.shadow[name] + (1.0 - self.decay) * param.data
            self.shadow[name] = new_avg.clone()

    @torch.no_grad()
    def apply_shadow(self):
        self.backup = {}
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.backup[name] = param.data.clone()
                param.data = self.shadow[name].clone()

    @torch.no_grad()
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                param.data = self.backup[name].clone()
        self.backup = {}


def linear_combination(x, y, epsilon):
    return epsilon * x + (1 - epsilon) * y

def reduce_loss(loss, reduction='mean'):
    return loss.mean() if reduction == 'mean' else loss.sum() if reduction == 'sum' else loss

class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, epsilon: float = 0.1, reduction='mean'):
        super().__init__()
        self.epsilon = epsilon
        self.reduction = reduction

    def forward(self, preds, target):
        n = preds.size()[-1]
        log_preds = F.log_softmax(preds, dim=-1)
        loss = reduce_loss(-log_preds.sum(dim=-1), self.reduction)
        nll = F.nll_loss(log_preds, target, reduction=self.reduction)
        return linear_combination(loss / n, nll, self.epsilon)



class IP102Classifier(nn.Module):
    
    def __init__(
        self,
        model_name: str = 'efficientnetv2_rw_s.ra2_in1k',
        num_classes: int = 102,
        pretrained: bool = True,
        dropout: float = 0.2, 
        drop_path_rate: float = 0.2
    ):
        super().__init__()
        
        # Carica backbone con stochastic depth
        try:
            self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg',
            drop_path_rate=drop_path_rate
            )
        except:
            print(" drop_path_rate non supportato")
            self.backbone = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
            )
        
        # get number of features
        num_features = getattr(self.backbone, 'num_features', None)

        if num_features is None:
            with torch.no_grad():
                dummy = torch.randn(1, 3, 384, 384)
                out = self.backbone(dummy)
                num_features = out.shape[1]
        
        # Classifier più robusto con BatchNorm
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(num_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )
        
        print(f" Modello: {model_name}")
        print(f" Features: {num_features}, Dropout: {dropout}, Drop Path: {drop_path_rate}")
    
    def forward(self, x):
        features = self.backbone(x)
        out = self.classifier(features)
        return out
    
    def freeze_backbone(self):
        # Congela backbone per fine-tuning graduale
        for param in self.backbone.parameters():
            param.requires_grad = False
        print(" Backbone congelato")
    
    def unfreeze_backbone(self):
        # Scongela backbone
        for param in self.backbone.parameters():
            param.requires_grad = True
        print(" Backbone scongelato")

class Trainer:
    
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        criterion,
        optimizer,
        scheduler,
        accum_steps=1,
        device: str = 'cuda',
        save_dir: str = './checkpoints',
        log_dir: str = './logs',
        use_mixup: bool = False,
        use_cutmix: bool = True,
        use_amp: bool = True,
        use_ema: bool = True,
        ema_decay: float = 0.999,
        mixup_prob: float = 0.5,
        mixup_alpha: float = 0.4,
        cutmix_alpha: float = 1.0,
        grad_clip: float = 5.0
    ):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.accum_steps = accum_steps
        self.device = device
        
        self.use_mixup = use_mixup
        self.use_cutmix = use_cutmix
        self.mixup_prob = mixup_prob
        self.mixup_alpha = mixup_alpha
        self.cutmix_alpha = cutmix_alpha
        self.grad_clip = grad_clip

        self.use_amp = use_amp
        self.scaler = GradScaler()

        self.use_ema = use_ema
        self.ema = EMA(self.model, decay=ema_decay) if self.use_ema else None
        
        self.save_dir = Path(save_dir)
        self.log_dir = Path(log_dir)
        self.save_dir.mkdir(parents=True, exist_ok=True)
        self.log_dir.mkdir(parents=True, exist_ok=True)
        
        self.history = {
            'train_loss': [], 'train_acc': [],
            'val_loss': [], 'val_acc': [],
            'val_balanced_acc': [], 'val_f1': [],
            'test_loss': None, 'test_acc': None,
            'test_balanced_acc': None, 'test_f1': None,
            'lr': [], 
        }
        
        self.best_val_acc = 0.0
        self.best_balanced_acc = 0.0
        self._patience_counter = 0


    def tta_forward(self, images):

        flips = [
            images,
            torch.flip(images, dims=[3])
        ]

        logits_sum = 0

        for f in flips:
            with autocast(enabled=True):
                out = self.model(f.float())
            logits_sum += out    

        return logits_sum / len(flips)

    
    
    def train_epoch(self, epoch: int) -> Tuple[float, Optional[float]]:
   
        self.model.train()

        correct = 0
        total = 0
        running_loss = 0.0

        clean_running_loss = 0.0
        clean_samples = 0

        
        pbar = tqdm(self.train_loader, desc=f'Epoch {epoch} [TRAIN]')
        
        for batch_idx, (images, labels) in enumerate(pbar):
            images = images.to(self.device, non_blocking=True)
            labels = labels.to(self.device, non_blocking=True)
            
            will_mix = (random.random() < self.mixup_prob) and (self.use_mixup or self.use_cutmix)
            
            if will_mix:
                if self.use_mixup and self.use_cutmix:
                    if random.random() < 0.5:
                        images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=self.mixup_alpha)
                    else:
                        images, labels_a, labels_b, lam = cutmix_data(images, labels, alpha=self.cutmix_alpha)
                elif self.use_mixup:
                    images, labels_a, labels_b, lam = mixup_data(images, labels, alpha=self.mixup_alpha)
                else:
                    images, labels_a, labels_b, lam = cutmix_data(images, labels, alpha=self.cutmix_alpha)

            if (batch_idx % self.accum_steps) == 0:
                self.optimizer.zero_grad(set_to_none=True)
            
            with autocast(enabled=self.use_amp):
                outputs = self.model(images)
                if will_mix:
                    loss = mixup_criterion(self.criterion, outputs, labels_a, labels_b, lam)
                else:
                    loss = self.criterion(outputs, labels)

                    clean_running_loss += loss.item() * images.size(0)
                    clean_samples += images.size(0)

            loss = loss / self.accum_steps

            if self.use_amp:
                # backward AMP
                self.scaler.scale(loss).backward()

                # grad clipping va fatto dopo unscale
                if self.grad_clip and self.grad_clip > 0.:
                    self.scaler.unscale_(self.optimizer)
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)

                if (batch_idx % self.accum_steps) == (self.accum_steps - 1):
                    self.scaler.step(self.optimizer)
                    self.scaler.update()

            else:
                loss.backward()
                
                if self.grad_clip and self.grad_clip > 0.:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.grad_clip)
                    
                if (batch_idx % self.accum_steps) == (self.accum_steps - 1):    
                    self.optimizer.step()

            if self.use_ema:
                self.ema.update()

            running_loss += (loss.item() * self.accum_steps) * images.size(0)

            if not will_mix:
                preds = torch.argmax(outputs, dim=1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        epoch_loss = running_loss / len(self.train_loader.dataset)
        clean_loss = clean_running_loss / clean_samples if clean_samples > 0 else None
        epoch_acc = correct / total if total > 0 else None

        return epoch_loss, epoch_acc, clean_loss
    
    @torch.no_grad()
    def validate(self, epoch: int) -> Dict[str, object]:

        if self.use_ema and self.ema is not None:
            self.ema.apply_shadow()
        
        self.model.eval()
        running_loss = 0.0
        all_preds = []
        all_labels = []
        
        pbar = tqdm(self.val_loader, desc=f'Epoch {epoch} [VAL]')

        
        for images, labels in pbar:
            images = images.to(self.device, non_blocking=True)
            labels = labels.to(self.device,non_blocking=True)
            
            try:
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
            except Exception as e:
                print(f" Batch saltato per errore: {e}")
                continue
            
            running_loss += loss.item() * images.size(0)
            
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.detach().cpu().numpy())
            all_labels.extend(labels.detach().cpu().numpy())

        # ripristina pesi dopo la validazione
        if self.use_ema and self.ema is not None:
            self.ema.restore()
        
        val_loss = running_loss / len(self.val_loader.dataset)
        val_acc = accuracy_score(all_labels, all_preds)
        val_balanced_acc = balanced_accuracy_score(all_labels, all_preds)
        val_f1_macro = f1_score(all_labels, all_preds, average='macro')
        val_f1_weighted = f1_score(all_labels, all_preds, average='weighted')
        
        return {
            'loss': val_loss,
            'accuracy': val_acc,
            'balanced_accuracy': val_balanced_acc,
            'f1_macro': val_f1_macro,
            'f1_weighted': val_f1_weighted,
            'preds': all_preds,
            'labels': all_labels
        }

    @torch.no_grad()
    def test(self, test_loader, use_tta=True):
        print(" TEST SET EVALUATION")

        # Usa EMA per il test
        if self.use_ema and self.ema is not None:
            self.ema.apply_shadow()

        self.model.eval()
        
        all_preds = []
        all_labels = []
        running_loss = 0.0

        for images, labels in tqdm(test_loader, desc='[TEST]'):
            images = images.to(self.device, non_blocking=True)
            labels = labels.to(self.device, non_blocking=True)

            if use_tta:
                with torch.cuda.amp.autocast(enabled=self.use_amp):
                    outputs = self.tta_forward(images)
            else:
                with torch.cuda.amp.autocast(enabled=self.use_amp):
                    outputs = self.model(images)

            loss = self.criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
    
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        test_loss = running_loss / len(test_loader.dataset)
        test_acc = accuracy_score(all_labels, all_preds)
        test_balanced_acc = balanced_accuracy_score(all_labels, all_preds)
        test_f1_macro = f1_score(all_labels, all_preds, average='macro')

        print(f"\nTest Loss: {test_loss:.4f}")
        print(f"Test Acc:  {test_acc:.4f}")
        print(f"Balanced:  {test_balanced_acc:.4f}")
        print(f"F1 macro:  {test_f1_macro:.4f}")

        # Confusion matrix test
        self.plot_confusion_matrix(all_labels, all_preds, epoch="TEST")

        self.history['test_loss'] = test_loss
        self.history['test_acc'] = test_acc
        self.history['test_balanced_acc'] = test_balanced_acc
        self.history['test_f1'] = test_f1_macro
        self.save_history()

        return {
            'loss': test_loss,
            'accuracy': test_acc,
            'balanced_accuracy': test_balanced_acc,
            'f1_macro': test_f1_macro,
            'preds': all_preds,
            'labels': all_labels
        }
    
    
    def train(
        self,
        num_epochs: int = 50,
        early_stopping_patience: int = 15,
        freeze_epochs: int = 5,
        save_best_only: bool = False
    ):
        print(f"Epochs: {num_epochs}, Early Stop Patience: {early_stopping_patience}")
        print(f"Freeze Backbone: {freeze_epochs} epochs")
        print(f"Mixup: {self.use_mixup}, CutMix: {self.use_cutmix}, mix_prob: {self.mixup_prob}")
        print(f"Gradient Clipping: {self.grad_clip}")


        # Freeze backbone initially
        if freeze_epochs > 0:
                self.model.freeze_backbone()


        patience_counter = 0
        start_time = time.time()
        
        for epoch in range(1, num_epochs + 1):
            
            if freeze_epochs > 0 and epoch == freeze_epochs + 1:

                if hasattr(self.model, 'unfreeze_backbone'):
                    self.model.unfreeze_backbone()
                else:
                    for p in self.model.parameters():
                        p.requires_grad = True

                backbone_params = []
                head_params = []
                
                for name, p in self.model.named_parameters():
                    if "classifier" in name:
                        head_params.append(p)
                    else:
                        backbone_params.append(p)

                self.optimizer = optim.AdamW(
                    [
                        {"params": backbone_params, "lr": 1e-5}, 
                        {"params": head_params, "lr": 1e-4},     
                    ],
                    weight_decay=1e-4,
                )

                self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                    self.optimizer,
                    T_max=num_epochs - epoch,
                    eta_min=1e-7                                                            
                )
                
                # reset scaler per sicurezza
                self.scaler = GradScaler(enabled=self.use_amp)
                print(" Backbone sbloccato + optimizer/scheduler aggiornati")
                
            
            train_loss, train_acc, clean_loss = self.train_epoch(epoch)
            val_metrics = self.validate(epoch)

            # Scheduler step (safe call)
            if self.scheduler is not None:
                try:
                    self.scheduler.step()
                except TypeError:
                    try:
                        self.scheduler.step(epoch)
                    except:
                        pass
            
          
            current_lrs = [param_group['lr'] for param_group in self.optimizer.param_groups]
            
            self.history['lr'].append(current_lrs)

            self.history['train_loss'].append(train_loss)
            self.history.setdefault('clean_train_loss', []).append(clean_loss)
            self.history['val_loss'].append(val_metrics['loss'])

            self.history['train_acc'].append(train_acc)
            self.history['val_acc'].append(val_metrics['accuracy'])

            self.history['val_balanced_acc'].append(val_metrics['balanced_accuracy'])
            self.history['val_f1'].append(val_metrics['f1_macro']) 

            # Print metriche
            
            overfit_gap = train_acc - val_metrics['accuracy']

            print(f"Epoch {epoch}/{num_epochs}")

            if train_acc is not None:
                print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
            else:
                print(f"Train Loss: {train_loss:.4f} | Train Acc: n/a (mixing used)")
                
            print(f"Val Loss:   {val_metrics['loss']:.4f} | Val Acc:   {val_metrics['accuracy']:.4f}")
            print(f"Balanced Acc: {val_metrics['balanced_accuracy']:.4f} | F1: {val_metrics['f1_macro']:.4f}")
            print(f" Overfit Gap: {overfit_gap:.4f} ({overfit_gap*100:.1f}%)")
            print(f"LR: {current_lrs[0]:.6f}")
            
            is_best = val_metrics['balanced_accuracy'] > self.best_balanced_acc
            if is_best:
                self.best_balanced_acc = val_metrics['balanced_accuracy']
                self.best_val_acc = val_metrics['accuracy']
                print(f" Best model - Balanced Acc: {self.best_balanced_acc:.4f}")
                patience_counter = 0
            else:
                patience_counter += 1
            
            self._patience_counter = patience_counter
            self.save_checkpoint(epoch, is_best=is_best)
            
            self.save_history()
            self.plot_training_curves()
            
            if is_best:
                self.plot_confusion_matrix(val_metrics['labels'], val_metrics['preds'], epoch)
            
            if patience_counter >= early_stopping_patience:
                print(f"\n Early stopping at epoch {epoch}")
                break
            
            print(f"Patience: {patience_counter}/{early_stopping_patience}")
        
        total_time = time.time() - start_time
        print(f"\n COMPLETATO in {total_time/60:.2f} min")
        print(f"Best Val Acc: {self.best_val_acc:.4f}, Balanced: {self.best_balanced_acc:.4f}\n")
        
        # Salvataggio finale
        self.save_history()

        self.plot_training_curves()
    
    def save_checkpoint(self, epoch: int, is_best: bool = False):
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scaler_state': self.scaler.state_dict() if self.use_amp else None,
            'ema_state': (self.ema.shadow if (self.use_ema and self.ema is not None) else None),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_val_acc': self.best_val_acc,
            'best_balanced_acc': self.best_balanced_acc,
            'history': self.history,
            'patience_counter': 0 if is_best else getattr(self, '_patience_counter', 0)
        }
        
        # Salva ultimo checkpoint (per resume)
        last_checkpoint_path = self.save_dir / 'last_checkpoint.pth'
        torch.save(checkpoint, last_checkpoint_path)
        print(f" Checkpoint salvato: epoch {epoch}")
        
        # Salva best model separatamente
        if is_best:
            best_checkpoint_path = self.save_dir / 'best_model.pth'
            torch.save(checkpoint, best_checkpoint_path)
            print(f" Best model aggiornato!")
        
        # Salva backup ogni 5 epochs
        if epoch % 5 == 0:
            backup_path = self.save_dir / f'checkpoint_epoch_{epoch}.pth'
            torch.save(checkpoint, backup_path)
            print(f" Backup salvato: epoch {epoch}")
    
    def save_history(self):
        with open(self.log_dir / 'history.json', 'w') as f:
            json.dump(self.history, f, indent=2)
    
    def plot_confusion_matrix(self, labels, preds, epoch):

        labels = np.array(labels)
        preds = np.array(preds)
        
        num_classes = self.model.classifier[-1].out_features

        if num_classes <= 20:
            class_list = np.arange(num_classes)
            cm = confusion_matrix(labels, preds, labels=class_list)

            plt.figure(figsize=(12, 10))
            sns.heatmap(cm, annot=False, fmt='d', cmap='Blues',
                        xticklabels=class_list, yticklabels=class_list)

        else:
            unique, counts = np.unique(labels, return_counts=True)
            top_classes = unique[np.argsort(counts)[-20:]]

            mask = np.isin(labels, top_classes)
            cm = confusion_matrix(labels[mask], preds[mask], labels=top_classes)

            plt.figure(figsize=(12, 10))
            sns.heatmap(cm, annot=False, fmt='d', cmap='Blues',
                        xticklabels=top_classes, yticklabels=top_classes)

        plt.title(f'Confusion Matrix - Epoch {epoch}')
        plt.tight_layout()
        plt.savefig(self.log_dir / f'confusion_matrix_ep{epoch}.png', dpi=150)
        plt.close()
    
    def plot_training_curves(self):
        if len(self.history['train_loss']) == 0:
            return
        
        clean_available = 'clean_train_loss' in self.history and any(
        x is not None for x in self.history['clean_train_loss']
        )

        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        epochs = range(1, len(self.history['train_loss']) + 1)

        if clean_available:
            axes[0, 0].plot(epochs, self.history['clean_train_loss'], 
                            label='Clean Train Loss (no mixup)', color='green')
        
        axes[0, 0].plot(epochs, self.history['val_loss'], label='Val', color="red")
        axes[0, 0].set_title('Loss'); axes[0, 0].legend()

        # Accuracy
        axes[0, 1].plot(epochs, self.history['train_acc'], label='Train')
        axes[0, 1].plot(epochs, self.history['val_acc'], label='Val')
        axes[0, 1].set_title('Accuracy'); axes[0, 1].legend()

        # Overfitting gap
        gap = [t - v for t, v in zip(self.history['train_acc'], self.history['val_acc'])]
        axes[1, 0].plot(epochs, gap)
        axes[1, 0].axhline(y=0, linestyle='--', alpha=0.5)
        axes[1, 0].set_title('Overfitting Gap (Train - Val)')

        # Metrics
        axes[1, 1].plot(epochs, self.history['val_balanced_acc'], label='Balanced Acc Val')
        axes[1, 1].plot(epochs, self.history['val_f1'], label='F1 Macro Val')

        axes[1, 1].set_title('Validation Metrics'); axes[1, 1].legend()

        plt.tight_layout()
        plt.savefig(self.log_dir / 'training_curves.png', dpi=300)
        plt.close()

def get_dataloaders(
    train_dir: str,
    val_dir: str,
    test_dir: str,
    batch_size: int = 16,
    img_size: int = 224,
    augment_level: str = 'heavy',
    use_weighted_sampling: bool = True,
    num_workers: int = 4
):
    # transforms
    mean = [0.485, 0.456, 0.406]
    std  = [0.229, 0.224, 0.225]

    if augment_level == 'heavy':
        train_tf = T.Compose([
            T.RandomResizedCrop(img_size, scale=(0.7, 1.0)),
            T.RandomHorizontalFlip(),
            T.RandomApply([T.ColorJitter(0.4,0.4,0.4,0.1)], p=0.8),
            T.ToTensor(),
            T.Normalize(mean=mean, std=std),
            T.RandomErasing(p=0.15)
        ])
    else:
        train_tf = T.Compose([
            T.Resize(int(img_size*1.15)),
            T.CenterCrop(img_size),
            T.ToTensor(),
            T.Normalize(mean=mean, std=std)
        ])

    val_tf = T.Compose([
        T.Resize(int(img_size*1.1)),
        T.CenterCrop(img_size),
        T.ToTensor(),
        T.Normalize(mean=mean, std=std)
    ])

    train_ds = datasets.ImageFolder(train_dir, transform=train_tf)
    val_ds = datasets.ImageFolder(val_dir, transform=val_tf)
    test_ds = datasets.ImageFolder(test_dir, transform=val_tf)

    
    if use_weighted_sampling:
        class_counts = np.bincount([y for _, y in train_ds.samples])
        weights = 1.0 / (class_counts + 1e-6)
        sample_weights = torch.tensor([weights[y] for _, y in train_ds.samples], dtype=torch.float)
        
        sampler = torch.utils.data.WeightedRandomSampler(sample_weights, len(sample_weights))
        
        train_loader = DataLoader(
            train_ds, 
            batch_size=batch_size, 
            sampler=sampler, 
            num_workers=num_workers, 
            pin_memory=True,
            drop_last=True
        )    
    else:
        train_loader = DataLoader(
            train_ds,
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_workers,
            pin_memory=True
            )

    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    return train_loader, val_loader, test_loader, train_ds



def main():
    CONFIG = {
        # Paths
        'train_dir': '/kaggle/input/ip02-dataset/classification/train',
        'val_dir': '/kaggle/input/ip02-dataset/classification/val',
        'test_dir': '/kaggle/input/ip02-dataset/classification/test',
        'save_dir': '/kaggle/working/checkpoints',
        'log_dir': '/kaggle/working/logs',
        
        # Model
        'model_name': 'efficientnetv2_rw_s.ra2_in1k',
        'num_classes': 102,
        'pretrained': True,
        'dropout': 0.2,  
        'drop_path_rate': 0.2, 
        
        # Training
        'batch_size': 32, 
        'img_size': 384,  
        'num_epochs': 25,
        'learning_rate': 1e-4,
        'weight_decay': 1e-4,
        'augment_level': 'heavy',
        'use_weighted_sampling': True,
        
        # Regularization
        'label_smoothing': 0.1,
        'use_mixup': True,
        'use_cutmix': True,
        'mixup_alpha': 0.1,
        'cutmix_alpha': 0.5,
        'mixup_prob': 0.2,
        'grad_clip': 1.0,
        'freeze_backbone_epochs': 3,
        'early_stopping_patience': 15,
        
        # Hardware
        'num_workers': 4,
        'device': 'cuda' if torch.cuda.is_available() else 'cpu'
    }
    
    print("⚙️  CONFIGURAZIONE")
    for key, value in CONFIG.items():
        print(f"{key:30s}: {value}")
    
    # Data loaders
    train_loader, val_loader, test_loader, train_ds = get_dataloaders(
        train_dir=CONFIG['train_dir'],
        val_dir=CONFIG['val_dir'],
        test_dir=CONFIG['test_dir'],
        batch_size=CONFIG['batch_size'],
        img_size=CONFIG['img_size'],
        augment_level=CONFIG['augment_level'],
        use_weighted_sampling=CONFIG['use_weighted_sampling'],
        num_workers=CONFIG['num_workers']
    )
    
    # Model
    model = IP102Classifier(
        model_name=CONFIG['model_name'],
        num_classes=CONFIG['num_classes'],
        pretrained=CONFIG['pretrained'],
        dropout=CONFIG['dropout'],
        drop_path_rate=CONFIG['drop_path_rate']
    )
    

    if CONFIG["label_smoothing"] > 0:
        criterion = LabelSmoothingCrossEntropy(epsilon=CONFIG["label_smoothing"])
    else:
        criterion = nn.CrossEntropyLoss()

    base_lr = CONFIG['learning_rate'] if CONFIG['learning_rate'] is not None else 1e-5
    optimizer = optim.AdamW(
        model.parameters(),
        lr=base_lr,
        weight_decay=CONFIG['weight_decay']
    )

    # scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=CONFIG["num_epochs"] - CONFIG['freeze_backbone_epochs'],
    eta_min=1e-7
    )


    # Trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=CONFIG['device'],
        save_dir=CONFIG['save_dir'],
        log_dir=CONFIG['log_dir'],
        use_mixup=CONFIG['use_mixup'],
        use_cutmix=CONFIG['use_cutmix'],
        mixup_prob=CONFIG['mixup_prob'],
        mixup_alpha=CONFIG['mixup_alpha'],
        cutmix_alpha=CONFIG['cutmix_alpha'],
        grad_clip=CONFIG['grad_clip'],
        use_amp=True,
        use_ema=True,
        ema_decay=0.999
    )
    
    # Training
    trainer.train(
        num_epochs=CONFIG['num_epochs'],
        early_stopping_patience=CONFIG['early_stopping_patience'],
        freeze_epochs=CONFIG['freeze_backbone_epochs']
    )

    # Test
    results = trainer.test(test_loader, use_tta=True)
    
    print("\n Training completato!")
    print(results)


if __name__ == "__main__":
    main()